# axialize scans

In [3]:
from pathlib import Path
import pandas as pd
import numpy as np
import re
from distutils.spawn import find_executable
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('max_colwidth',500)

In [5]:
analysis_version = "2018_07_03"
project_dir_absolute = Path('/tmp/euler_number_prediction')
%pwd
%cd {project_dir_absolute}
%pwd

/private/tmp/euler_number_prediction


'/private/tmp/euler_number_prediction'

In [36]:
orig_scans  = project_dir_absolute / 'orig_scans'
output_dir = project_dir_absolute / 'axialized_scans'
afni_dir = Path('~').expanduser() / 'abin'
if not afni_dir.exists():

        afni_path = Path(find_executable('afni')).parent.parent
        if not afni_path.exists():
            raise EnvironmentError
        else: 
            afni_dir = afni_path.parent
        
input_template  = afni_dir / 'MNI_2009c.nii.gz'

In [32]:
scans = pd.DataFrame({'scan_path':[p.as_posix() for p in orig_scans.glob('*.nii.gz')]})

In [33]:
scans.head()


,scan_path
0,/tmp/euler_number_prediction/orig_scans/sub-1289_ses-1_acq-mprage_run-001_T1w.nii.gz
1,/tmp/euler_number_prediction/orig_scans/sub-0414_ses-1_acq-mprage_run-002_T1w.nii.gz
2,/tmp/euler_number_prediction/orig_scans/sub-1223_ses-1_acq-mprage_run-001_T1w.nii.gz
3,/tmp/euler_number_prediction/orig_scans/sub-1360_ses-1_acq-mprage_run-002_T1w.nii.gz
4,/tmp/euler_number_prediction/orig_scans/sub-1491_ses-1_acq-mprage_run-002_T1w.nii.gz


In [34]:
scan_path = scans.scan_path.iloc[0]

scan_path

'/tmp/euler_number_prediction/orig_scans/sub-1289_ses-1_acq-mprage_run-001_T1w.nii.gz'

In [46]:
# I/O path, same as above; just need the "proc" dirs now
path_P_ss = orig_scans

# reference anatomical volumes to which we axialize
here       = project_dir_absolute
ref_t1w    = input_template
# ref_t1w_wt = assert False

cmd = f"""\
fat_proc_axialize_anat\
    -inset  {scan_path}\
    -prefix {output_dir / Path(scan_path).name.split('.')[0]}\
    -mode_t1w\
    -refset          {ref_t2w}\
    -out_match_ref\
"""
cmd

'fat_proc_axialize_anat    -inset  /tmp/euler_number_prediction/orig_scans/sub-1289_ses-1_acq-mprage_run-001_T1w.nii.gz    -prefix /tmp/euler_number_prediction/axialized_scans/sub-1289_ses-1_acq-mprage_run-001_T1w    -mode_t1w    -refset          /tmp/euler_number_prediction/mni_icbm152_t2_relx_tal_nlin_sym_09a_ACPCE.nii.gz    -out_match_ref'

In [47]:
!{cmd}

++ Start script version: 3.9
++ 3dinfo: AFNI version=AFNI_18.1.09 (Apr 30 2018) [64-bit]
** AFNI converts NIFTI_datatype=4 (INT16) in file /tmp/euler_number_prediction/orig_scans/sub-1289_ses-1_acq-mprage_run-001_T1w.nii.gz to FLOAT32
     Warnings of this type will be muted for this session.
     Set AFNI_NIFTI_TYPE_WARN to YES to see them all, NO to see none.
118
++ 3dinfo: AFNI version=AFNI_18.1.09 (Apr 30 2018) [64-bit]
** AFNI converts NIFTI_datatype=4 (INT16) in file /tmp/euler_number_prediction/orig_scans/sub-1289_ses-1_acq-mprage_run-001_T1w.nii.gz to FLOAT32
     Warnings of this type will be muted for this session.
     Set AFNI_NIFTI_TYPE_WARN to YES to see them all, NO to see none.
++ Found input file:   /tmp/euler_number_prediction/orig_scans/sub-1289_ses-1_acq-mprage_run-001_T1w.nii.gz
++ 3dinfo: AFNI version=AFNI_18.1.09 (Apr 30 2018) [64-bit]
** ERROR (nifti_image_read): failed to find header file for '/tmp/euler_number_prediction/mni_icbm152_t2_relx_tal_nlin_sym_09a_AC